<a href="https://colab.research.google.com/github/katterine2558/mri-tumor-detection-cnn/blob/main/mri_tumor_detection_cnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MRI Tumor Detection

Este notebook tiene como objetivo desarrollar y evaluar un modelo de red neuronal convolucional (CNN) para la detección de tumores cerebrales en imágenes de resonancia magnética (MRI).
El proyecto busca aplicar técnicas de procesamiento de imágenes médicas y aprendizaje profundo para clasificar las imágenes en diferentes tipos de tumores, contribuyendo así al apoyo del diagnóstico médico asistido por inteligencia artificial.

Se incluyen etapas de preprocesamiento de datos, diseño del modelo, entrenamiento y validación, así como el análisis de métricas y visualizaciones que permitan interpretar el desempeño del modelo.


## Configuración de entorno (Colab/GPU)

En esta sección se realiza la configuración inicial del entorno de ejecución para optimizar el entrenamiento del modelo dentro de Google Colab Pro.
El objetivo es garantizar que los recursos de hardware (GPU y memoria) se utilicen de forma eficiente, lo que permite acelerar significativamente los cálculos durante el entrenamiento de la red neuronal.

Primero, se limpia la sesión anterior para liberar memoria y evitar conflictos con modelos previamente cargados.
Luego, se verifica la disponibilidad de una GPU y se fuerza su uso mediante TensorFlow, configurando además el crecimiento dinámico de memoria para prevenir errores por saturación.

Finalmente, se activa el entrenamiento de precisión mixta (mixed precision), una técnica que combina cálculos en float16 y float32 para mejorar la velocidad sin afectar la precisión del modelo.
Con esta configuración, el entorno queda preparado para trabajar con el máximo rendimiento disponible.

In [ ]:
# ==========================================================
# CONFIGURACIÓN DE ENTORNO COLAB PRO (GPU + OPTIMIZACIONES)
# ==========================================================
import os, gc, psutil, tensorflow as tf
from tensorflow.keras import mixed_precision

#Limpiza sesión anterior
gc.collect()
tf.keras.backend.clear_session()

#Información del hardware
try:
    gpu_info = !nvidia-smi --query-gpu=name,memory.total --format=csv,noheader
    print("GPU detectada:", gpu_info[0])
except:
    print("No se detectó GPU, verifica que el acelerador esté activado en Entorno > Configuración de ejecución.")

#Forzar uso de GPU
physical_gpus = tf.config.list_physical_devices('GPU')
if physical_gpus:
    print(f"TensorFlow detecta {len(physical_gpus)} GPU(s): {physical_gpus}")
    try:
        for gpu in physical_gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except:
        pass
else:
    print("No hay GPU activa, se usará CPU (entrenamiento más lento).")

#Precisión mixta para iniciar el entrenamiento
mixed_precision.set_global_policy('mixed_float16')
tf.debugging.set_log_device_placement(False)
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
print("Política de precisión:", mixed_precision.global_policy())

#Información de recursos
print(f"Memoria RAM disponible: {psutil.virtual_memory().total / 1e9:.2f} GB")
print("Entorno configurado correctamente para máximo rendimiento.\n")

## Importación de librerías y configuración inicial

En esta sección se importan todas las librerías necesarias para el desarrollo del proyecto de detección de tumores cerebrales mediante imágenes de resonancia magnética (MRI).
Estas librerías permiten realizar tareas de procesamiento de imágenes, manipulación de datos, construcción del modelo, entrenamiento y evaluación de resultados.
Además, se establecen parámetros base como las rutas de trabajo (`base_dir` y `split_dir`), el tamaño de lote (`batch_size`), el tamaño de las imágenes (`image_size`) y la semilla aleatoria (`seed`) para garantizar la reproducibilidad de los resultados. Finalmente, se verifica la disponibilidad del dispositivo GPU, se activa la política de precisión mixta y se ajustan las configuraciones del entorno para optimizar el rendimiento durante el entrenamiento del modelo.

In [ ]:
#==========================================================
# LIBRERÍAS
#==========================================================
from google.colab import files
#!pip install -q kaggle #Descomentar si no está instalado Kaggle en el entorno
import os
import random
import matplotlib.pyplot as plt
import cv2
import numpy as np
from collections import Counter
import shutil
from sklearn.model_selection import train_test_split
import tensorflow as tf
import pandas as pd
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Input, Dense, Conv2D, MaxPooling2D, Add, Activation,GlobalAveragePooling2D , Multiply, Reshape, DepthwiseConv2D, BatchNormalization ,GlobalAveragePooling2D, Resizing, Rescaling, RandomBrightness, RandomContrast, RandomRotation, RandomZoom, BatchNormalization, Dropout
from tensorflow.keras.utils import plot_model
from tensorflow.keras.optimizers import Adam, AdamW
from tensorflow.keras.initializers import HeNormal
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.metrics import AUC
!pip install keras-tuner --upgrade
from keras_tuner.tuners import RandomSearch
from tensorflow.keras import mixed_precision
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
from sklearn.metrics import roc_auc_score, roc_curve, auc
from sklearn.preprocessing import label_binarize
import json
!nvidia-smi

#==========================================================
# CONFIGURACIÓN INICIAL
#==========================================================

#Ruta base
base_dir = '/content/mri_data'
#Ruta para almacenar split
split_dir = "/content/mri_data_split"
#Semilla
seed = 0;
#Tamaño del batch
batch_size = 16
#Image size
image_size = (260, 260)


## Descarga dataset
Se descarga el dataset de MRI cerebrales desde Kaggle mediante la API, organizando las imágenes por categoría.
Luego, se listan las clases disponibles y se muestran ejemplos representativos para una inspección visual inicial.

In [ ]:
#==========================================================
# DOWNLOAD DATASET
#==========================================================
def download_mri_data(base_dir:str):

  files.upload() #APIKey de Kaggle

  !mkdir -p ~/.kaggle
  !mv kaggle.json ~/.kaggle/
  !chmod 600 ~/.kaggle/kaggle.json

  # Descargar el dataset
  !mkdir -p /content/mri_data
  !kaggle datasets download -d rm1000/brain-tumor-mri-scans -p /content/mri_data --unzip

  # Listar categorías
  categories = os.listdir(base_dir)
  print("Categorías encontradas:", categories)

  # Crear grilla 3x4: 3 muestras aleatorias por cada clase (una columna por clase)
  n_rows = 3
  n_cols = len(categories)
  fig, axes = plt.subplots(n_rows, n_cols, figsize=(4*n_cols, 4*n_rows))

  for j, c in enumerate(categories):
      folder = os.path.join(base_dir, c)
      img_list = os.listdir(folder)
      for i in range(n_rows):
          # Selecciona una imagen aleatoria de la clase
          img_name = random.choice(img_list)
          img_path = os.path.join(folder, img_name)
          img = cv2.imread(img_path)
          if img is None:
              continue
          img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

          ax = axes[i, j]
          ax.imshow(img)
          if i == 0:  # Solo la primera fila lleva el título
              ax.set_title(c.capitalize(), fontsize=12, fontweight='bold')
          ax.axis('off')

  plt.tight_layout()
  plt.show()

#Descarga el conjunto de datos y visualiza
download_mri_data(base_dir)

## Análisis exploratorio y preprocesamiento de imágenes

### Distribución de clases

En esta etapa se analiza la distribución del número de imágenes por clase dentro del conjunto de datos.
Este gráfico de barras permite identificar posibles desequilibrios entre categorías, lo cual es importante antes de entrenar el modelo, ya que una distribución desigual podría afectar el desempeño y la generalización de la red neuronal.

In [ ]:
# ==========================================================
# DISTRIBUCIÓN DE CLASES
# ==========================================================
def plot_class_distribution(base_dir: str):

    class_counts = {}

    # Recorre cada carpeta (clase)
    for cls in sorted(os.listdir(base_dir)):
        cls_path = os.path.join(base_dir, cls)
        if not os.path.isdir(cls_path):
            continue
        n_images = len([
            f for f in os.listdir(cls_path)
            if f.lower().endswith(('.png', '.jpg', '.jpeg'))
        ])
        class_counts[cls] = n_images

    # --- Gráfico de barras ---
    plt.figure(figsize=(6,4))
    bars = plt.bar(class_counts.keys(), class_counts.values(), color="#828282")
    plt.xlabel("Clase")
    plt.ylabel("Número de imágenes")
    plt.xticks(rotation=15)
    plt.ylim(0,2200)

    # Etiquetas encima de cada barra
    for bar in bars:
        yval = bar.get_height()
        plt.text(bar.get_x() + bar.get_width()/2, yval + 20, str(yval),
                 ha='center', va='bottom', fontsize=9)

    plt.tight_layout()
    plt.show()

    return class_counts

# Ejecutar función
plot_class_distribution(base_dir)


### Exploración de resoluciones en las imágenes

En esta sección se analizan las dimensiones (ancho y alto) de las imágenes del conjunto de datos.
Los histogramas permiten observar la variabilidad en las resoluciones originales, lo que ayuda a determinar la necesidad de aplicar un redimensionamiento uniforme antes del entrenamiento del modelo y asegurar una entrada consistente a la red neuronal.

In [ ]:
# ==========================================================
# HISTOGRAMAS DE ANCHOS Y ALTOS DE LAS IMÁGENES
# ==========================================================
def plot_image_size_distribution(base_dir: str, sample_per_class: int = 1000):
    """
    Calcula y grafica la distribución de anchos y altos de las imágenes
    muestreando hasta sample_per_class por clase.
    """
    widths, heights = [], []

    for cls in sorted(os.listdir(base_dir)):
        cls_path = os.path.join(base_dir, cls)
        if not os.path.isdir(cls_path):
            continue

        # limitar muestras por clase (para rapidez)
        img_files = [
            f for f in os.listdir(cls_path)
            if f.lower().endswith(('.png', '.jpg', '.jpeg'))
        ][:sample_per_class]

        for fname in img_files:
            img_path = os.path.join(cls_path, fname)
            img = cv2.imread(img_path)
            if img is None:
                continue
            h, w = img.shape[:2]
            widths.append(w)
            heights.append(h)

    # --- Estadísticas ---
    print(f"Total de imágenes analizadas: {len(widths)}")
    print(f"• Ancho promedio:  {np.mean(widths):.1f}px | Rango: {min(widths)}–{max(widths)}")
    print(f"• Alto  promedio:  {np.mean(heights):.1f}px | Rango: {min(heights)}–{max(heights)}")

    # --- Subplots ---
    fig, axes = plt.subplots(1, 2, figsize=(10, 4))

    axes[0].hist(widths, bins=20, color="#4B9CD3", edgecolor='black', alpha=0.8)
    axes[0].set_title("a) Distribución de anchos")
    axes[0].set_xlabel("Ancho (px)")
    axes[0].set_ylabel("Frecuencia")

    axes[1].hist(heights, bins=20, color="#E69138", edgecolor='black', alpha=0.8)
    axes[1].set_title("b) Distribución de altos")
    axes[1].set_xlabel("Alto (px)")
    axes[1].set_ylabel("Frecuencia")

    plt.tight_layout()
    plt.show()

# Ejecutar función
plot_image_size_distribution(base_dir)


### Visualización de Data Augmentation

En esta sección se muestran ejemplos ilustrativos de las transformaciones de data augmentation implementadas dentro del modelo.
Estas operaciones —como volteo horizontal, rotación, zoom y ajuste de contraste— permiten aumentar la variabilidad del conjunto de entrenamiento, ayudando al modelo a generalizar mejor y evitar el sobreajuste.

In [ ]:
# ==========================================================
# VISUALIZACIÓN DEL DATA AUGMENTATION (EJEMPLOS ILUSTRATIVOS)
# ==========================================================
def visualize_data_augmentation(base_dir: str, image_size=(224,224)):
    """
    Muestra una grilla con una imagen original y las transformaciones de
    data augmentation aplicadas (flip, rotación, zoom, contraste)
    para cada clase.
    """

    # Capas de augmentación (idénticas a las del modelo)
    aug_flip = tf.keras.layers.RandomFlip("horizontal")
    aug_rotation = tf.keras.layers.RandomRotation(0.08)
    aug_zoom = tf.keras.layers.RandomZoom(0.1)
    aug_contrast = tf.keras.layers.RandomContrast(0.2)

    transformations = [
        ("Original", lambda x: x),
        ("Flip", aug_flip),
        ("Rotación", aug_rotation),
        ("Zoom", aug_zoom),
        ("Contraste", aug_contrast),
    ]

    # Clases ordenadas
    class_names = sorted([
        d for d in os.listdir(base_dir)
        if os.path.isdir(os.path.join(base_dir, d))
    ])

    n_rows = len(class_names)
    n_cols = len(transformations)

    fig, axes = plt.subplots(n_rows, n_cols, figsize=(3.5*n_cols, 3.5*n_rows))
    if n_rows == 1:
        axes = np.expand_dims(axes, axis=0)  # asegura 2D

    for i, cls in enumerate(class_names):
        # Seleccionar una imagen aleatoria
        cls_path = os.path.join(base_dir, cls)
        candidates = [f for f in os.listdir(cls_path) if f.lower().endswith(('.png','.jpg','.jpeg'))]
        if not candidates:
            continue
        img_path = os.path.join(cls_path, random.choice(candidates))

        # Cargar y preparar la imagen
        img = tf.keras.utils.load_img(img_path, target_size=image_size)
        img_array = tf.keras.utils.img_to_array(img)
        img_array = np.array(img_array, dtype=np.float32) / 255.0
        img_batch = tf.expand_dims(img_array, 0)

        for j, (name, transform) in enumerate(transformations):
            ax = axes[i, j]
            if name == "Original":
                aug_img = img_array
            else:
                aug_img = transform(img_batch, training=True)[0].numpy().astype(np.float32)

            ax.imshow(np.clip(aug_img, 0, 1))
            if i == 0:
                ax.set_title(name, fontsize=12, fontweight='bold')
            if j == 0:
                ax.set_ylabel(cls.capitalize(), fontsize=12, rotation=0, labelpad=60)
            ax.axis("off")

    plt.tight_layout()
    plt.show()

# Ejecutar
visualize_data_augmentation(base_dir, image_size=(image_size[0], image_size[1]))

In [ ]:
# ==========================================================
# TRAIN-VAL-TEST DIVISION (70-20-10)
# ==========================================================
def traint_val_test_split(base_dir:str, split_dir:str):
    # Crear carpetas destino
    for split in ['train', 'val', 'test']:
        for cls in os.listdir(base_dir):
            os.makedirs(os.path.join(split_dir, split, cls), exist_ok=True)

    # Ratios
    train_ratio = 0.7
    val_ratio = 0.2
    test_ratio = 0.1

    # Procesar cada clase
    for cls in os.listdir(base_dir):
        cls_path = os.path.join(base_dir, cls)
        images = [f for f in os.listdir(cls_path) if f.lower().endswith(('.png','.jpg','.jpeg'))]

        # Dividir train / val / test
        train_files, temp_files = train_test_split(images, test_size=(1-train_ratio), random_state=seed)
        val_size = val_ratio / (val_ratio + test_ratio)
        val_files, test_files = train_test_split(temp_files, test_size=(1-val_size), random_state=seed)

        # Copiar archivos
        for fname in train_files:
            shutil.copy(os.path.join(cls_path, fname), os.path.join(split_dir, 'train', cls, fname))
        for fname in val_files:
            shutil.copy(os.path.join(cls_path, fname), os.path.join(split_dir, 'val', cls, fname))
        for fname in test_files:
            shutil.copy(os.path.join(cls_path, fname), os.path.join(split_dir, 'test', cls, fname))

    print("División completada con semilla =", seed)
    print("Estructura creada en:", split_dir)

    for split in ['train', 'val', 'test']:
        print(f"\n * {split.upper()}")
        for cls in os.listdir(os.path.join(split_dir, split)):
            count = len(os.listdir(os.path.join(split_dir, split, cls)))
            print(f"  {cls}: {count} imágenes")


# Ejecutar división solo una vez
traint_val_test_split(base_dir, split_dir)

# ==========================================================
# CARGA DE LOS DATASETS
# ==========================================================
train_ds_raw = tf.keras.utils.image_dataset_from_directory(
    f"{split_dir}/train",
    image_size=image_size,
    batch_size=batch_size,
    shuffle=True,
    seed=seed
)

val_ds_raw = tf.keras.utils.image_dataset_from_directory(
    f"{split_dir}/val",
    image_size=image_size,
    batch_size=batch_size,
    shuffle=True,
    seed=seed
)

test_ds_raw = tf.keras.utils.image_dataset_from_directory(
    f"{split_dir}/test",
    image_size=image_size,
    batch_size=batch_size,
    shuffle=False,
    seed=seed
)

# Guarda los nombres de clase ANTES de optimizar los datasets
class_names = test_ds_raw.class_names
print("Clases detectadas:", class_names)

# ==========================================================
# OPTIMIZACIÓN DE LOS DATASETS
# ==========================================================
AUTOTUNE = tf.data.AUTOTUNE

def prepare_dataset(ds, shuffle=False):
    if shuffle:
        ds = ds.shuffle(buffer_size=1000)
    ds = ds.prefetch(AUTOTUNE)
    return ds

train_ds = prepare_dataset(train_ds_raw, shuffle=True)
val_ds   = prepare_dataset(val_ds_raw)
test_ds  = prepare_dataset(test_ds_raw)

print("Datasets listos y optimizados para entrenamiento.")

In [ ]:
# ==========================================================
# ACTIVACIÓN MISH
# ==========================================================
@tf.keras.utils.register_keras_serializable()
def mish(x):
    return x * tf.math.tanh(tf.math.softplus(x))


# ==========================================================
# BLOQUE MBConv MEJORADO (con DropConnect)
# ==========================================================
def MBConvMRI(inputs, out_channels, expand_ratio, stride, se_ratio=0.25, drop_rate=0.2):
    in_channels = inputs.shape[-1]
    shortcut = inputs
    x = inputs

    expanded = int(in_channels * expand_ratio)
    if expand_ratio != 1:
        x = layers.Conv2D(expanded, 1, padding='same', use_bias=False, kernel_initializer=HeNormal())(x)
        x = layers.BatchNormalization()(x)
        x = layers.Activation(mish)(x)

    # Depthwise separable
    x = layers.DepthwiseConv2D(3, strides=stride, padding='same', use_bias=False)(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation(mish)(x)

    # Squeeze and Excitation
    se = layers.GlobalAveragePooling2D()(x)
    se = layers.Reshape((1,1,expanded))(se)
    se = layers.Conv2D(int(expanded * se_ratio), 1, activation=mish, padding='same')(se)
    se = layers.Conv2D(expanded, 1, activation='sigmoid', padding='same')(se)
    x = layers.Multiply()([x, se])

    # Proyección
    x = layers.Conv2D(out_channels, 1, padding='same', use_bias=False, kernel_initializer=HeNormal())(x)
    x = layers.BatchNormalization()(x)

    # Residual con DropConnect
    if stride == 1 and in_channels == out_channels:
        if drop_rate > 0:
            x = layers.Dropout(drop_rate)(x)
        x = layers.Add()([x, shortcut])
    return x


# ==========================================================
# MODELO EfficientMRI-Net v2
# ==========================================================
def build_efficient_mri_net_v2(num_classes=4, input_shape=(image_size[0], image_size[1], 3)):
    inputs = layers.Input(shape=input_shape)

    # -----------------------------
    # Data augmentation integrado
    # -----------------------------
    x = layers.RandomFlip("horizontal")(inputs)
    x = layers.RandomRotation(0.08)(x)
    x = layers.RandomZoom(0.1)(x)
    x = layers.RandomContrast(0.2)(x)
    x = layers.Rescaling(1./255)(x)

    # -----------------------------
    # Bloque inicial
    # -----------------------------
    x = layers.Conv2D(48, 3, strides=2, padding='same', use_bias=False, kernel_initializer=HeNormal())(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation(mish)(x)

    # -----------------------------
    # Bloques MBConv
    # -----------------------------
    x = MBConvMRI(x, 16, 1, 1, drop_rate=0.1)
    x = MBConvMRI(x, 24, 3, 2, drop_rate=0.1)
    x = MBConvMRI(x, 24, 3, 1, drop_rate=0.1)
    x = MBConvMRI(x, 40, 4, 2, drop_rate=0.15)
    x = MBConvMRI(x, 40, 4, 1, drop_rate=0.15)
    x = MBConvMRI(x, 80, 4, 2, drop_rate=0.2)
    x = MBConvMRI(x, 80, 4, 1, drop_rate=0.2)
    x = MBConvMRI(x, 112, 4, 1, drop_rate=0.25)
    x = MBConvMRI(x, 160, 4, 2, drop_rate=0.25)
    x = MBConvMRI(x, 320, 4, 1, drop_rate=0.3)

    # -----------------------------
    # Head final
    # -----------------------------
    x = layers.Conv2D(1280, 1, padding='same', use_bias=False, kernel_initializer=HeNormal())(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation(mish)(x)

    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dropout(0.4)(x)
    outputs = layers.Dense(num_classes, activation='softmax', dtype='float32')(x)

    model = Model(inputs, outputs, name="EfficientMRI_Net_v2")
    return model


# ==========================================================
# COMPILACIÓN
# ==========================================================
model = build_efficient_mri_net_v2(num_classes=4, input_shape=(image_size[0], image_size[1],3))
optimizer = AdamW(learning_rate=8e-4, weight_decay=1e-5)
loss_fn = SparseCategoricalCrossentropy()

model.compile(
    optimizer=optimizer,
    loss=loss_fn,
    metrics=['accuracy']
)

# ==========================================================
# CALLBACKS
# ==========================================================
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.6, patience=5, min_lr=1e-5, verbose=1)
early_stop = EarlyStopping(monitor='val_loss', patience=6, restore_best_weights=True, verbose=1)
checkpoint = ModelCheckpoint('EfficientMRI_Net_v2.keras', monitor='val_accuracy', save_best_only=True, verbose=1)

model.summary()


# ==========================================================
# ENTRENAMIENTO O CARGA DEL MODELO
# ==========================================================
history = None

if os.path.exists('EfficientMRI_Net_v2.keras'):
    print("Cargando modelo desde el checkpoint...")
    model.load_weights('EfficientMRI_Net_v2.keras')
    print("Pesos cargados correctamente.")
else:
    print("Iniciando entrenamiento desde cero...")
    history = model.fit(
        train_ds,
        validation_data=val_ds,
        epochs=80,
        batch_size=batch_size,
        callbacks=[reduce_lr, early_stop, checkpoint],
        verbose=1
    )


In [ ]:
# ==========================================================
# GGUARDA LA HISTORIA
# ==========================================================
# Guardar la historia completa como JSON
with open('history_efficient_mri_net_v2.json', 'w') as f:
        json.dump(history.history, f)

# ==========================================================
# GRÁFICAS DE ENTRENAMIENTO Y VALIDACIÓN
# ==========================================================
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(1, len(acc) + 1)

plt.figure(figsize=(14,5))

# Accuracy
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, 'b-', label='Entrenamiento')
plt.plot(epochs_range, val_acc, 'r--', label='Validación')
plt.title('Evolución del Accuracy')
plt.xlabel('Época')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True, linestyle='--', alpha=0.5)

# Loss
plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, 'b-', label='Entrenamiento')
plt.plot(epochs_range, val_loss, 'r--', label='Validación')
plt.title('Evolución de la Pérdida (Loss)')
plt.xlabel('Época')
plt.ylabel('Loss')
plt.legend()
plt.grid(True, linestyle='--', alpha=0.5)

plt.show()

In [ ]:
# ==========================================================
# EVALUACIÓN FINAL EN TEST
# ==========================================================
test_loss, test_acc = model.evaluate(test_ds, verbose=1)
print(f"\n✅ Test Accuracy: {test_acc:.4f}")
print(f"📉 Test Loss: {test_loss:.4f}")


In [ ]:
# ==========================================================
# REPORTE DE MATRIZ DE CONFUSIÓN
# ==========================================================
y_true = np.concatenate([y for x, y in test_ds], axis=0)
y_pred_probs = model.predict(test_ds)
y_pred = np.argmax(y_pred_probs, axis=1)

print("\n🧾 Reporte de Clasificación:\n")
print(classification_report(y_true, y_pred, target_names=class_names, digits=2))

cm = confusion_matrix(y_true, y_pred)
plt.figure(figsize=(7,6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=class_names, yticklabels=class_names)
plt.title("Matriz de Confusión - EfficientMRI-Net")
plt.xlabel("Predicción")
plt.ylabel("Real")
plt.show()


In [ ]:
# ==========================================================
# CÁLCULO DE AUC MULTICLASE
# ==========================================================
# Convertimos las etiquetas verdaderas a formato binarizado (one-hot)
y_true_bin = label_binarize(y_true, classes=np.arange(len(class_names)))

# AUC por clase y promedio
auc_macro = roc_auc_score(y_true_bin, y_pred_probs, multi_class='ovr', average='macro')
auc_weighted = roc_auc_score(y_true_bin, y_pred_probs, multi_class='ovr', average='weighted')

print(f"\nAUC promedio (macro): {auc_macro:.4f}")
print(f"AUC promedio (ponderado): {auc_weighted:.4f}\n")


# ==========================================================
# CURVAS ROC (opcional, para visualizar por clase)
# ==========================================================
plt.figure(figsize=(8,6))
for i, class_name in enumerate(class_names):
    fpr, tpr, _ = roc_curve(y_true_bin[:, i], y_pred_probs[:, i])
    roc_auc = auc(fpr, tpr)
    plt.plot(fpr, tpr, lw=2, label=f'{class_name} (AUC = {roc_auc:.2f})')

plt.plot([0, 1], [0, 1], color='gray', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('Tasa de Falsos Positivos (FPR)')
plt.ylabel('Tasa de Verdaderos Positivos (TPR)')
plt.title('Curvas ROC - EfficientMRI-Net')
plt.legend(loc='lower right')
plt.show()